In [1]:
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor  
import joblib

# Load the dataset from the Pickle file
with open('simulated_dataset.pkl', 'rb') as f:
    df = pickle.load(f)

# Display a preview of the dataframe to check the data
print(df.head())



   entry log  Lap Motorsport_Type    Team    Driver                     Track  \
0        0.0    1             DTM  Team_A  Driver_1  Nürburgring Nordschleife   
1        1.0    1             DTM  Team_A  Driver_1  Nürburgring Nordschleife   
2        2.0    1             DTM  Team_A  Driver_1  Nürburgring Nordschleife   
3        3.0    1             DTM  Team_A  Driver_1  Nürburgring Nordschleife   
4        4.0    1             DTM  Team_A  Driver_1  Nürburgring Nordschleife   

   lap_time  Throttle  Brake  Steering_Position  ...  Tire_wear  \
0       1.0      0.70    0.0           0.666705  ...   0.011969   
1       2.0      0.00    1.0          -0.511976  ...   0.024995   
2       3.0      0.62    0.0          -0.271574  ...   0.030101   
3       4.0      1.00    0.0           0.390544  ...   0.039367   
4       5.0      1.00    0.0           0.347772  ...   0.047961   

   cumilative_Tire_Wear  Driving_Style  force_on_tire    Event  \
0              0.153938         Normal   128

In [ ]:
df['Driving_Style'] = df['Driving_Style'].map({'Aggressive': 2, 'Normal': 1, 'Conservative': 0})

# Features and targets
X = df[["Speed", "Throttle", "Ambient_Temperature", "Driving_Style", "Lap", "lap_time", "force_on_tire"]]
y = df[["Tire_degradation", "Tire_wear"]]

# Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train separate models for each target (Tire_degradation, Tire_wear)
models = {}
for target in y.columns:
    regressor = RandomForestRegressor(n_estimators=100, random_state=42)
    regressor.fit(X_train, y_train[target])
    models[target] = regressor

# Save the trained models to a file using joblib
joblib.dump(models, "models.pkl")

#print out the performance on the test set (for debugging)
for target in y.columns:
    score = models[target].score(X_test, y_test[target])
    print(f"R-squared score for {target}: {score:.4f}")


R-squared score for Tire_degradation: 0.9991


In [ ]:
from flask import Flask, request, jsonify, render_template_string
import joblib
import numpy as np

app = Flask(__name__)

# Load pre-trained models
models = joblib.load("models.pkl")

# HTML Form Template as a string 
html_form = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Tire Prediction</title>
</head>
<body>
    <h2>Tire Degradation and Wear Prediction</h2>
    <form action="/predict" method="POST">
        <label for="Speed">Speed (km/h):</label><br>
        <input type="text" id="Speed" name="Speed" required><br><br>
        
        <label for="Pressure">Pressure (psi):</label><br>
        <input type="text" id="Pressure" name="Pressure" required><br><br>
        
        <label for="Temperature">Temperature (°C):</label><br>
        <input type="text" id="Temperature" name="Temperature" required><br><br>

        <label for="DrivingStyle">Driving Style (e.g., Aggressive, Normal, Conservative):</label><br>
        <input type="text" id="DrivingStyle" name="DrivingStyle" required><br><br>

        <label for="NumberOfLaps">Number of Laps:</label><br>
        <input type="text" id="NumberOfLaps" name="NumberOfLaps" required><br><br>

        <label for="Distance">Distance (km):</label><br>
        <input type="text" id="Distance" name="Distance" required><br><br>

        <label for="ForceOnTire">Force on Tire (N):</label><br>
        <input type="text" id="ForceOnTire" name="ForceOnTire" required><br><br>

        <input type="submit" value="Predict">
    </form>

    <div id="result">
        {% if prediction %}
        <h3>Predictions:</h3>
        <p><strong>Tire Degradation:</strong> {{ prediction['Tire_degradation'] }}</p>
        <p><strong>Tire Wear:</strong> {{ prediction['Tire_wear'] }}</p>
        {% endif %}
    </div>
</body>
</html>
"""


@app.route('/')
def home():
    return render_template_string(html_form)

@app.route('/predict', methods=['POST'])
def predict():
    # Get data from the form
    speed = float(request.form['Speed'])
    pressure = float(request.form['Pressure'])
    temperature = float(request.form['Temperature'])
    driving_style = request.form['DrivingStyle']
    number_of_laps = int(request.form['NumberOfLaps'])
    distance = float(request.form['Distance'])
    force_on_tire = float(request.form['ForceOnTire'])

    # Convert categorical input (DrivingStyle) to numeric encoding (e.g., Aggressive=2, Normal=1, Conservative=0)
    driving_style_map = {'Aggressive': 2, 'Normal': 1, 'Conservative': 0}
    driving_style_encoded = driving_style_map.get(driving_style, 1)  # Default to Normal if not found

    # Prepare feature array, including new inputs
    features = np.array([[speed, pressure, temperature, driving_style_encoded, number_of_laps, distance, force_on_tire]])

    # Predict using the models
    predictions = {target: model.predict(features)[0] for target, model in models.items()}

    # Render the result on the HTML page
    return render_template_string(html_form, prediction=predictions)

if __name__ == '__main__':
     app.run(debug=True, use_reloader=False)


: 